In [ ]:
knitr::opts_chunk$set(echo = TRUE)
options(yaml.eval.expr = TRUE)


In [ ]:
# Necessary to embed the youtube videos.
library(vembedr)


## Introduction



In [ ]:
vembedr::embed_url("https://www.youtube.com/watch?v=Jc87nRBSPWQ")



# Section 1: Exploring, renaming, and filtering data



In [ ]:
vembedr::embed_url("https://www.youtube.com/watch?v=mK7vYaetHI8")



## Workshop Setup

I'm first going to load the couple of packages that we will need for this workshop. I've included a little bit of information about each of their uses in a comment below. Running the code chunk below will load the desired packages if you have already installed them, and the nice thing is, it will also automatically install any needed packages if you have not yet done so. Basically, this is possible through creating an object containing the names of the desired packages ("wrangling_packages"), creating an object containing the names of your existing packages ("packages"), and using a function to install any package names that are listed among the desired packages, but not among existing packages.


In [ ]:
wrangling_packages <- c("data.table", "effects", "lme4", "lubridate", "modelsummary", 
                        "psych", "qdap", "reshape", "rJava", "rlang", "skimr", 
                        "syuzhet", "tidytext", "tidyverse", "utils", "yaml")

##brief explanation of packages##
#tidyverse is for wrangling and contains ggplot2, tibble, purrr, dplyr, tidyr, stringr, readr, and forcats
#data.table is for reading large data objects
#skimr and psych are for basic statistics
#rlang and rershape are for basic wrangling
#utils, modelsummary, and yaml are for R notebook functionality
#lubridate is for updating date/time stamps
#qdap, rJava, syuzhet, and tidytext are for text analysis and cleaning
#lme4 is for linear regression exercise

packages <- rownames(installed.packages())
p_to_install <- wrangling_packages[!(wrangling_packages %in% packages)]

if(length(p_to_install) > 0){
  install.packages(p_to_install)
}

lapply(wrangling_packages, library, character.only = TRUE)


NOTE: If you receive an error stating: ```error: JAVA_HOME cannot be determined from the Registry```, try [this troubleshooting page to resolve the issue](https://www.r-statistics.com/2012/08/how-to-load-the-rjava-package-after-the-error-java_home-cannot-be-determined-from-the-registry/).

Now let's clean our environment of the variables we won't need 


In [ ]:
rm(p_to_install, packages, wrangling_packages)




We're next going to specify the working directory to wherever you have this workshop stored locally.


In [ ]:
#check current working directory
getwd()

#change the directory to where you have the COG2022_DataWranglinginR.csv saved
wd <- "YOUR/PATH/HERE"
setwd(wd)

#we won't be using the variable wd any longer
rm(wd)


In [ ]:
#check current working directory
getwd()

#change the directory to where you have the COG2022_DataWranglinginR.csv saved
wd <- "C:/Users/tui81100/Documents/GitHub/cog_summer_workshops_2022/data-wrangling-in-r/"
setwd(wd)

#we won't be using the variable wd any longer
rm(wd)


## About the data

For this workshop, we will be using publicly-available New York Times Cooking comment data (found via Kaggle, which is a great source for test datasets if you're interested). The data was collected via New York Times's API (source below). This is the sort of data you might be using if you use web-scraped or publiclly-available social media data in your work (think Tweets, comments, online reviews), and there are all sorts of things you can do with data like this especially since there tend to be so many observations and they are completely naturalistic. 
* [Kaggle source](https://www.kaggle.com/datasets/michaelruddy/new-york-times-recipe-comments)
* [New York Times API](https://developer.nytimes.com)

## Import raw data

We're first going to import the New York Times Cooking comment data as a dataframe. There are a few ways to do this that we'll go through. You can use the "read.csv" function from the ```utils``` package, which is likely the most common way to load .csv data. However, You can also use the ```fread``` function from the ```data.table``` package, which is much better for datasets of this size, as we'll see (thank you Billy for introducing me to this one)! 

Note that we first list the package that it's from followed by two colons and then the function. This is to let R know where to find the function. This usually isn't necessary, but doing this will allow you to call a function without necessarily loading it into our environment with ```library()``` or, as in our case, resolve issues between packages when two packages have a similarly named function. It just tells R which package we want R to pull *Function X* from.

Also note that the dataframe starts as a zipped file. This is because the raw data is too large to be hosted by Github. You will need to extract the file first. Usually, this is an option in your right-click menu. 


In [ ]:
nyt_raw <- data.table::fread("df.csv", 
                 na.strings = c('', 'NA'))


We're going to save the raw data to a dataframe called "nyt_raw", which we can access anytime we need the raw data going forward. To get a sense of our data structure, we'll use the ```head``` function to show a snippet of the first few rows of the dataframe (6 rows by default). We can adjust the number of rows shown with the ```n = 20``` command within the head function.



In [ ]:
head(nyt_raw, n = 20) #this will show a snippet of the first 20 rows of the data file



The data I imported here is the raw data file downloaded straight from the Kaggle page. As we can see from the snippet above, it's not very well-organized, which is where this workshop should come in handy! We'll walk through organization steps now.

## Data Organization
Based on the New York Times Cooking API and the data source on Kaggle, it looks like the columns in our dataset are organized as follows. Italicized column names are those we will be examining today: 

* V1: the index of the comment within a specific recipe 
* *commentID: the comment's unique identifying number* 
* status: whether or not NYT has approved the comment 
* commentSequence: the comment's sequence 
* *userID: the ID number of the user who wrote the comment* 
* userDisplayName: first name or handle selected by user who wrote the comment
* userLocation: location of the user (locked)
* userTitle: the user's title, if they have one (e.g., "food critic"; locked) 
* userURL: the user's NYT cooking profile URL (locked) 
* picURL: the URL of the photo included in the comment (locked) 
* commentTitle: the "subject line"/title of the user's comment 
* *commentBody: the comment text* 
* *createDate: unix time stamp of comment creation date* 
* updateDate: unix time stamp of any comment updates 
* approveDate: unix time stamp of comment approval date 
* *recommendations: number of other users who liked a specific comment* 
* *replyCount: number of replies to a particular comment* 
* replies: replies written to each specific comment 
* editorsSelection: whether or not the comment has been selected as an "Editor's Selection" comment 
* parentID: if reply, comment ID for the original comment (locked) 
* parentUserDisplay: if reply, user ID for original comment (locked) 
* depth: comment depth 
* commentType: original comment vs. reporter reply 
* trusted: whether or not the comment is trusted 
* recommendedFlag: whether or not the comment has a recommended flag 
* permID: permanent comment ID - 
* isAnonymous: whether or not the comment is anonymous 
* *recipe_id: ID number for the recipe commented on* 
* *recipe_name: name of the recipe commented on*

## Examining raw data structure

We will now explore our dataset using a couple of Base R commands.


In [ ]:
#copying the data to a new object so that we can retain the raw data separately
nyt <- nyt_raw


First, we're going to get a sense of the data using a couple of basic Base R commands. This will tell us about the structure of the dataset, or the way in which R categorizes each of the variables (i.e., numeric, character, etc.). This is important to keep in mind because R will treat variables differently based upon how they are structured.



In [ ]:
str(nyt) #shows us what kinds of variables we have



Next, we'll use the ```names()``` function to examine the column names of our dataframe, which essentially function as the names of our variables. 



In [ ]:
names(nyt) #shows column names



We can look at how frequently each unique value appeared within any given column using the ```tables()``` function and specifying the column as the object of the function. Let's look at how many replies each entry got.



In [ ]:
table(nyt$replyCount)



We can also use it with qualitative variables like recipe names.



In [ ]:
table(nyt$recipe_name)




. . . and we can clean this table up a little bit by converting it into a dataframe


In [ ]:
data.frame(table(nyt$recipe_name))



Next, we can get descriptive statistics for individual numeric variables.



In [ ]:
mean(nyt$replyCount)
sd(nyt$replyCount)


We can also use visualizations to understand our variables of interest.



In [ ]:
hist(nyt$replyCount)
plot(nyt$replyCount)


Checking our data for quality assurance could clearly become a lengthy process depending on the number of variables you're looking at! Here are a few commands to make data exploration more concise:

NOTE: The ```datasummary_skim``` function output may appear empty in Markdown, but text is just white and can be seen if highlighted.


In [ ]:
#skim the entire dataframe using the skimr package
skim(nyt)
datasummary_skim(nyt,) #this makes a nice table for R-markdown


## Troubleshooting

From here on out, if you have any questions on how a specific function works or is formatted, the R Studio help window will help you out. To search a function, simply write a question mark (?) followed by the function in your console.


In [ ]:
#?[FUNCTION] #use this to learn more about how a function works in the "help" panel to the right
?hist()


## Check your work

I will also be using the ```head()``` function from the ```utils``` package to periodically check work at each step in the process.


In [ ]:
head(nyt, n = 10) #specify number of rows desired using "n =" (default is 6 rows)



## Renaming columns

It's clear that the raw data we downloaded is pretty messy, and is going to require some wrangling on our end. First off, some of our columns are not named very descriptively nor consistently. There are generally two naming conventions used in R (and data science more generally): "columnName" or "column_name". We have a few columns that are each named using different conventions, which adds to the messiness. We're going to give our columns more consistent and descriptive names using the ```rename()``` function from the ```dplyr``` package.


In [ ]:
names(nyt)

# Rename individual columns using consistent naming conventions with rest of df
nyt <- dplyr::rename(nyt, 
                     c('recipeName' = 'recipe_name',
                       'recipeID' = 'recipe_id'))

# Or rename columns in a batch!
nyt <- rename_with(nyt, ~ (gsub("user", 
                                "author", 
                                .x, 
                                fixed = TRUE)))

#check work
head(nyt, n = 10)


## Selecting columns and filtering rows

It looks like there are a lot of extra columns we aren't interested in in this data. We can access the column names directly using the ```names``` function (above), and then use ```dplyr```'s select() function to choose the columns we would like to keep. I usually specify that I want to use ```dplyr```'s ```select()``` function instead of any select function from another package (for example, there's also one in the ```psych``` package that sometimes conflicts with ```dplyr```) by writing the package name and 2 colons before the function (```dplyr::select()```). 

We can also clean data up to use only rows that we are interested in using ```filter()``` or ```subset()``` functions. I'm first going to select all rows with "approved" comments by using the ```filter()``` function. It's worth noting that one can filter for rows containing values in a column that are equal to (```==```) or unequal to (```!=```) a specific reference value. You can do the same for numeric variables (as we'll see later on in the "arrange and sort" section), or could also select reference values that are greater than (```>```)/greater than or equal to (```>=```), or less than (```<```)/less than or equal to (```<=```) a specific reference value.


In [ ]:
#select by row: this will allow us to keep only comments that have been "approved"
nyt <- filter(nyt, status == "approved") #find status = approved
nrow(nyt) #looks like we didn't lose any rows, so they all were "approved"!

#select by column: this will allow us to choose only columns we'll need for the rest of our analyses
names(nyt)
nyt <- dplyr::select(nyt, c('commentID', 'commentBody', 'createDate', 
                            'recommendations', 'recipeName', 'recipeID', 
                            'replyCount', 'authorID', 'status'))

#remove an additional column
nyt <- dplyr::select(nyt, -c('status'))

#just for instructional purposes-- select columns with with certain titles using "matches" and "starts_with"
nyt_recipes <- dplyr::select(nyt, matches('recipe') | matches('comment'))
nyt_recipes <- dplyr::select(nyt, starts_with('recipe'))
nyt_IDs <- dplyr::select(nyt, ends_with('ID'))

#you can also use "select" to rearrange columns into a desired order:
nyt <- dplyr::select(nyt, c("recipeName", "recipeID"), everything())

head(nyt, n = 10)


## Using piping to make data wrangling more concise

By using piping through the ```magittr``` package, it's possible to execute all of what we worked though above in one code chunk! We will first save the raw data to a new dataframe called "nyt_concise", and execute a more concise version of the steps taken above to reach the same result we have in our current "nyt" dataframe. Keep in mind that there are multiple ways to do this-- for example, you could select the columns directly or using the ```matches``` argument as I do here.


In [ ]:
nyt_concise <- nyt_raw

nyt_concise <- nyt_concise %>% 
               filter(status == "approved") %>%
               dplyr::select(matches('recipe') | 
                            matches('commentID') | 
                            matches('commentBody') | 
                            matches('recommendations') | 
                            matches('createDate') | 
                            matches('replyCount')) %>%
               dplyr::rename(c('recipeName' = 'recipe_name',
                               'recipeID' = 'recipe_id'))

head(nyt_concise)


## Arrange

Let's say we wanted to have sort our data by date of the original comment. The comments come in unix date-time format, and we can use the as_datetime() function from the ```lubridate``` package to put the date into a readable format To arrange by date, we'll use the ```arrange()``` function from ```dplyr```, which arranges values in ascending order by default and can be reversed by adding the ````desc()``` argument.


In [ ]:
#replace UNIX timestamps with interpretable date/time
nyt$createDate <- as_datetime(nyt$createDate)

#arrange by date (in ascending order by default: earliest dates first)
nyt <- arrange(nyt, createDate)

#arrange by date in descending order: latest dates first
nyt <- arrange(nyt, desc(createDate))

#select only reviews from 2021
nyt <- filter(nyt, createDate >= '2021-01-01 00:00:00') #greater than or equal to operator
nrow(nyt) #down from 389,963 to 53,355 reviews after cutting out all before 2021

head(nyt, n = 10)


## Small Group Exercise 1: Making data wrangling more concise

How would we add the ```arrange()```, and ```filter()``` functions above into our "nyt_concise" dataframe?


In [ ]:
#changing the creation date to timedate format must be done separately
nyt_concise$createDate <-lubridate::as_datetime(nyt_concise$createDate)

nyt_concise <- nyt_concise %>%  #ADD HERE

head(nyt_concise)


[Click for solution](https://github.com/TU-Coding-Outreach-Group/cog_summer_workshops_2022/tree/main/data-wrangling-in-r/solutions/exercise1.R)

# Section 2: Mutating data, merging dataframes, exporting data


In [ ]:
vembedr::embed_url("https://www.youtube.com/watch?v=DG64hpm1yFc")



## Mutate and Transmute: create columns based on existing columns

To create new columns based on the content of other columns in a dataframe, we can use the ```mutate()``` function from the ```dplyr``` package. Using the ```transmute()``` function will achieve the same results as ```mutate()``` and is written the same way, but while ```mutate()``` will retain both the original and newly-created columns, ```transmute()``` keeps only the newly-created column. Here, we'll create a new column containing the number of words in each comment called "textLength" and will keep the reference column ("commentBody") by using ```mutate()```.


In [ ]:
### Create New Columns ###
#create a new column based on other columns
nyt <- mutate(nyt, textLength = qdap::word_count(nyt$commentBody)) #text length column

head(nyt, n = 10)


## Miscellaneous text cleaning

Since we have text data, I'm going to introduce a couple of functions for cleaning text data without getting too much into the details of Natural Language Processing (that workshop is in a few weeks, but feel free to check out this awesome online book called "Supervised Machine Learning for Text Analysis in R" by Emil Hvitfeldt & Julia Silge if you're interested: <https://smltar.com>). I'll first use the the ```str_trim()``` function from the ```stringr``` package to take out any extra characters encoded in the comment body, and will next create a dataframe called "nyt_text" containing only the comment body data. I will first tokenize the comments, next calculate sentiment scores using the "AFINN" dictionary, and lastly standardize these such that scores closer to +1 indicate stronger positive sentiment, while scores closer to -1 indicate stronger negative sentiment.


In [ ]:
#remove unwanted characters
nyt$commentBody <- str_trim(gsub("\n\n", "", nyt$commentBody)) 
nyt$commentBody <- str_trim(gsub("\n", "", nyt$commentBody)) 

#tokenize
nyt_text <- dplyr::select(nyt, c('commentBody', 'commentID'))
nyt_text <- nyt_text %>% unnest_tokens(word, commentBody) %>% #create tokens for each word
  anti_join(stop_words) #remove stop words using SMART lexicon
nyt_text <- nyt_text %>% group_by(commentID) %>% summarise(commentBody = paste0(word, collapse = ' ')) #collapse back into original reviews

#get sentiment scores for each comment
nyt_text$sentiment <- get_sentiment(nyt_text$commentBody, method = "afinn")
head(nyt_text)


In [ ]:
#standardize sentiment scores
nyt_text$sentiment <- scale(nyt_text$sentiment, center=T, scale=T)

#rename tokenized comment body
nyt_text <- rename(nyt_text, c('commentToken' = 'commentBody'))

head(nyt_text)


## Join dataframes

Lastly, we will walk through ```dplyr```'s functions for merging two dataframes together. We'll use ```left_join()``` so that all of the columns from our newly-cleaned nyt_text dataframe can be matched with columns in our cleaned nyt dataframe. There are several different ways this could be achieved using ```dplyr```'s join functions: ```left_join()``` (appends all rows in df y that match those in df x to df x), ```right_join()``` (appends all rows in df x that match df y to df y), ```inner_join()``` (appends all rows that match in df x and df y), and ```full_join()``` (appends all rows in df x and df y regardless of row match). These can each be helpful depending on the data structure you're hoping to achieve.


In [ ]:
#left_join to merge nyt and nyt_text dataframes
nyt <- left_join(nyt, nyt_text, by=c('commentID'))

head(nyt, n = 20)


## Saving your cleaned data

Especially for large datasets like this one, it can be costly for your computer's memory (and your time!) to rerun all of the cleaning steps above. By writing the cleaned data into a new csv file, you can load in your successfully wrangled data the next time you want to use it for analyses or visualizations.


In [ ]:
#if you don't want to rerun data cleaning steps, you can save and read in cleaned data directly
write.csv(nyt, "nyt_clean.csv") #this will save the cleaned file from above into a csv called "nyt_clean" in your working directory fodler


Congratulations, you now have the skills to wrangle any large and messy dataset you might encounter into a beautiful, usable format! It is important to note that this is not an exhaustive walkthrough of all wrangling techniques available to you both in terms of other packages or formats (e.g., Base R) nor in terms of ```dplyrv's wrangling functionality itself, but I hope it's been helpful for introducing a few new data wrangling techniques into your repertoire. For the end of thr workshop, I'm going to give a few exercises and practical examples to show where some of the techniques we walked through might come in handy and to encourage you to look into others.

# Section 3: Application

## Small Group Exercise 2: Review

For the second exercise, I'm going to ask you to use the functions above to answer a few specific questions about our final cleaned "nyt" dataset that might be similar to some exploratory steps you'd take on your own data.


In [ ]:
## How many rows and columns are in the dataset?

## What classes are the "commentBody" and "textLength" columns?

## Turn textLength into a numeric variable (suggestion: use as.numeric()).

## Use the skim() function to find the mean, SD, min and max for textLength, sentiment (Hint: we standardized sentiment earlier on, so M should be 0 and SD should be 1), and replyCount.

## Remove rows containing outliers for textLength, sentiment, and replyCount (greater or less than 3SD from the mean)

## Remove any rows with missing data. (Hint: use drop_na())

## Plot histograms of the sentiment, textLength, and replyCount columns.

## Using the select() function and datasummary_correlation() function from the modelsummary package, create an intercorrelation table of all numeric variables aside from date and ID variables.


[Click for solution](https://github.com/TU-Coding-Outreach-Group/cog_summer_workshops_2022/tree/main/data-wrangling-in-r/solutions/exercise2.R)

## Small Group Exercise 3 & 4: Wrangling Data for Specific Analyses

The last exercise is going to be a bit of advanced application of what we've walked through so far, and which might require a bit more exploration of possible solutions online. Remember, there can be more than one way to achieve the same result, so it's fine if your solution looks different than mine as long as the steps all make sense! If you're interested, I encourage you to look into different ways to achieve some of these same results using [the cheatsheets included in this workshop folder on Github](https://github.com/TU-Coding-Outreach-Group/cog_summer_workshops_2022/tree/main/data-wrangling-in-r/supplements), or search alternatives on Google or Stack Overflow. As a bonus, try to wrangle data as concisely as possible using the pipe functions!

1. Using a t-test via the ```t.test()``` function, compare comment sentiment for 2 different recipes. First, wrangle data into a usable format, and then perform the t-test. Feel free to use the help command in R ("?FUNCTION()) or the cheatsheets for any help. I've included suggested steps, but feel free to do whatever makes most sense to you.


In [ ]:
## Save nyt as new dataframe

## Call the recipeName function using the $ operator to choose recipe names

## Select needed columns

## Use subset() or filter() to select rows for 2 recipe names

## Use pivot_wider() (new function!) to transform data from long to wide format with recipe names as column names and data as sentiment

## Use t.test to compare sentiment for recipe 1 with recipe 2


[Click for solution](https://github.com/TU-Coding-Outreach-Group/cog_summer_workshops_2022/tree/main/data-wrangling-in-r/solutions/exercise3.R)

2. Using two simple linear regressions via the ```lm()``` function, examine a) whether replyCount is predicted by sentiment, and b) whether textLength predicts sentiment. First, wrangle data into a usable format, and then perform the regressions. Feel free to use the help command in R ("?FUNCTION()) or the cheatsheets for any help. I've included suggested steps, but feel free to do whatever makes most sense to you.


In [ ]:
## Save nyt as new dataframe

## Select needed columns 

## Standardize replyCount and textLength

## Use lm() and plot(effect()) to estimate linear regression models and visualize results



[Click for solution](https://github.com/TU-Coding-Outreach-Group/cog_summer_workshops_2022/tree/main/data-wrangling-in-r/solutions/exercise4.R)
